In [1]:
import pandas as pd
from sqlalchemy.engine import URL # ao usar pyodbc o pandas continuava dando aviso recomendando usar sqlalchemy, portanto mudei a biblioteca utilizada para a query
from sqlalchemy import create_engine, text
import boto3
import ast

In [2]:
secret_name = "database_tcepb"
region_name = "sa-east-1"

# Create a Secrets Manager client
session = boto3.session.Session()
client = session.client(
    service_name='secretsmanager',
    region_name=region_name
)

get_secret_value_response = client.get_secret_value(
    SecretId=secret_name
)
SERVER_TRIBUNAL = ast.literal_eval(get_secret_value_response["SecretString"])["host"] + ", " + ast.literal_eval(get_secret_value_response["SecretString"])["port"]
DB_TRIBUNAL = ast.literal_eval(get_secret_value_response["SecretString"])["dbname"]
USERNAME = ast.literal_eval(get_secret_value_response["SecretString"])["username"]
PASSWORD = ast.literal_eval(get_secret_value_response["SecretString"])["password"]

In [126]:
string_conexao = "DRIVER={SQL Server};" + f"SERVER={SERVER_TRIBUNAL};DATABASE={DB_TRIBUNAL};UID={USERNAME};PWD={PASSWORD}"
url_conexao = URL.create("mssql+pyodbc", query={"odbc_connect": string_conexao}) # cria uma URL de conexão para ser colocado em engine
engine = create_engine(url_conexao) # Documentation: https://docs.sqlalchemy.org/en/14/core/engines.html

In [127]:
cnpj = "'11308834000185'"

## Funções

In [128]:
def busca_sql(query: str) -> pd.DataFrame:
    """
    Recebe uma string como sql query e retorna do dataframe dessa busca.
    """
    with engine.begin() as conn:
        df = pd.read_sql_query(text(query), conn)
    return df

In [129]:
def comparando_dfs(df_raw: list[str], df_filtered: list[str]) -> list[str]:
    """Recebe a lista no primeiro argumento e retorna os valores que não estão contidos na segunda lista."""
    nao_passou_no_filtro = []
    for chave in df_raw:
        if chave not in df_filtered:
            nao_passou_no_filtro.append(chave)
    return nao_passou_no_filtro

## Notas Normais

### Queries

In [130]:
query_notas_ativas_raw = \
rf"""
SELECT
	DISTINCT chave.chave
FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_NFE AS nfe -- Juntando com a tabela NFE para, a partir dela, fazer ligação com a tabela DIM_DESTINATARIO_EMITENTE

INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE AS chave -- Juntando com tabela DIM_CHAVE para pegar a chave da nota fiscal
	ON nfe.id_chave = chave.id_chave

INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS emitente_destinatario -- Relação das notas fiscais com os dados de seus emitentes
	ON nfe.id_emitente = emitente_destinatario.id_destinatario_emitente

INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CPF_CNPJ AS cpf_cnpj -- Relação com tabela DIM_CPF_CNPJ para pegar o cnpj
	ON emitente_destinatario.id_cpf_cnpj = cpf_cnpj.id_cpf_cnpj

WHERE
	nfe.nota_cancelada = 0 AND
	nfe.id_finalidade = 1 AND
	cpf_cnpj.cnpj = {cnpj} AND
	chave.chave NOT IN (
		SELECT
			DISTINCT chave.chave
		FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_NFE AS nfe -- Juntando com a tabela NFE para, a partir dela, fazer ligação com a tabela DIM_DESTINATARIO_EMITENTE

		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE AS chave -- Juntando com tabela DIM_CHAVE para pegar a chave da nota fiscal
			ON nfe.id_chave = chave.id_chave

		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS emitente_destinatario -- Relação das notas fiscais com os dados de seus emitentes
			ON nfe.id_emitente = emitente_destinatario.id_destinatario_emitente

		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CPF_CNPJ AS cpf_cnpj -- Relação com tabela DIM_CPF_CNPJ para pegar o cnpj
			ON emitente_destinatario.id_cpf_cnpj = cpf_cnpj.id_cpf_cnpj

		WHERE
			nfe.nota_cancelada = 1 AND
			cpf_cnpj.cnpj = {cnpj}
	)
ORDER BY chave.chave;
"""

In [131]:
query_notas_ativas_filtered = \
rf"""
SELECT
	DISTINCT chave.chave
FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_PRODUTO AS produtos -- Carregando a tabela produtos que é o alvo da query
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.FATO_NFE AS nfe -- Juntando com a tabela NFE para, a partir dela, fazer ligação com a tabela DIM_DESTINATARIO_EMITENTE
	ON produtos.id_chave = nfe.id_chave
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE AS chave -- Juntando com tabela DIM_CHAVE para pegar a chave da nota fiscal
	ON produtos.id_chave = chave.id_chave
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS emitente -- Relação das notas fiscais com os dados de seus emitentes
	ON nfe.id_emitente = emitente.id_destinatario_emitente
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS destinatario -- Relação das notas fiscais com os dados de seus destinatarios
	ON nfe.id_destinatario = destinatario.id_destinatario_emitente
LEFT JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_ENDERECO as endereco -- Relação com DIM_ENDERECO para pegar o municipio de cada destinatario
	ON destinatario.id_endereco = endereco.id_endereco
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CPF_CNPJ AS cpf_cnpj -- Relação com tabela DIM_CPF_CNPJ para pegar o cnpj
	ON emitente.id_cpf_cnpj = cpf_cnpj.id_cpf_cnpj
INNER JOIN RF.dbo.t_pessoa_juridica AS pessoa_juridica -- Relação com banco de dados da Rceita Federal através do NPJ para filtras os códigos CNAE das empresas
	ON cpf_cnpj.cnpj = pessoa_juridica.cnpj
--------------------------------------------------------------------------------------------------------------------------------------------------------------
WHERE
--FILTRO PRODUTOS
	(
		-- Código para bebidas, líquidos alcoólicos e vinagres (Capítulo 22)
		produtos.codigo_ncm LIKE '22071%' OR
		produtos.codigo_ncm LIKE '22072%' OR
		-- Códigos NCM para produtos farmacêuticos (Capítulo 30)
		produtos.codigo_ncm LIKE '30%' OR
		-- Código Extratos tanantes e tintoriais (Capítulo 32)
		produtos.codigo_ncm LIKE '320416%' OR -- corantes reagentes e preparações à base desses corantes
		-- Código para produtos para fotografia e cinematografia (Capítulo 37)
		produtos.codigo_ncm LIKE '370110%' OR -- filme para raio x
		-- Código produtos químicos diversos (Capítulo 38)
		produtos.codigo_ncm LIKE '3821%' OR -- Meios de cultura preparados para o desenvolvimento e a manutenção de microrganismos 
		produtos.codigo_ncm LIKE '3822%' OR -- reagentes de diagnóstico ou de laboratório (e.g. analisador de PH, glicose liquida, fósforo UV, hemoglobina glicada, acido urico)
		-- Código NCM para produtos de borracha (Capítulo 40)
		produtos.codigo_ncm LIKE '40151%' OR
		-- Código para tecidos especiais, tecidos tufados, rendas, tapeçarias, passamanarias, bordados (Capítulo 58)
		produtos.codigo_ncm LIKE '5803%' OR -- Tecidos em ponto de gaze
		-- Código para Outros artefatos têxteis confeccionados (Capítulo 60)
		produtos.codigo_ncm LIKE '6307901%' OR -- Outros artefatos confeccionados de falso tecido (e.g. máscara)
		-- Código Reatores nucleares, caldeiras, máquinas, aparelhos e instrumentos mecânicos, e suas partes (Capítulo 84)
		produtos.codigo_ncm LIKE '842119%' OR -- Centrifugadores para laboratórios de análises, ensaios ou pesquisas científicas
		produtos.codigo_ncm LIKE '842191%' OR -- Partes de centrifugadores
		produtos.codigo_ncm LIKE '84798912' OR -- Distribuidores e doseadores de sólidos ou de líquidos  (e.g. micropipeta)
		-- Códigos NCM para instrumentos e aparelhos de óptica, de fotografia, de cinematografia, de medida, de controle ou de precisão.
		-- instrumentos e aparelhos médico-cirúrgicos suas partes e acessórios (Capítulo 90)
		produtos.codigo_ncm LIKE '9011%' OR --  Microscópios ópticos, incluindo os microscópios para fotomicrografia, cinefotomicrografia ou microprojeção.
		produtos.codigo_ncm LIKE '9012%' OR --  Microscópios, exceto ópticos; difratógrafos.
		produtos.codigo_ncm LIKE '9018%' OR -- instumentos e aparelhos para medicina, cirurgia, odontologia e veterinária
		produtos.codigo_ncm LIKE '9019%' OR -- aparelhos de mecanoterapia, aparelhos de massagem, de psicotécnica, de ozonoterapia...
		produtos.codigo_ncm LIKE '9020%' OR -- outros aparelhos respiratórios e máscaras contra gases
		produtos.codigo_ncm LIKE '9021%' OR -- Artigos e aparelhos ortopédicos
		produtos.codigo_ncm LIKE '9022%' OR -- Aparelhos de raios X e aparelhos que utilizem radiações alfa
		produtos.codigo_ncm LIKE '9027%' OR -- Instrumentos e aparelhos para análises físicas ou químicas
		-- Códigos para Obras diversas (Capítulo 96)
		produtos.codigo_ncm LIKE '9619%' OR --  Absorventes e tampões higiênicos, cueiros e fraldas para bebês e artigos higiênicos semelhantes, de qualquer matéria.
			-- FONTE: http://www5.sefaz.mt.gov.br/documents/6071037/6401784/Tabela+NCM+-+MDIC+atualizada.pdf/bc780e4b-fd2f-4312-879c-65d5fd1ff49d / Pág. 129
		produtos.id_medicamento != -1 -- possui id de medicamento válido
	) AND
--------------------------------------------------------------------------------------------------------------------------------------------------------------
--FILTRO NOTA FISCAL
	nfe.nota_cancelada = 0 AND -- Notas que não foram canceladas
	nfe.id_modelo = 2 AND -- Notas 55
	nfe.id_finalidade = 1 AND -- Notas que não possuem valor 4 (DEVOLUCAO DE MERCADORIA), nem valor -1 (finalidade nula) e nem valor -2 (TIPO INVALIDO CONSULTE NFE)
--------------------------------------------------------------------------------------------------------------------------------------------------------------
--FILTRO PESSOA JURIDICA 4645101
	pessoa_juridica.cnae_fiscal IN (2110600, 2121101, 2121102, 2121103, 2123800, 2312500, 3299099, 4618401,
									4644301, 4645101, 4664800, 4684299, 4771701, 4771702, 4771703, 4773300) -- Empresas farmoquimicas e farmaceuticas
-- Fonte: https://www.contabilizei.com.br/contabilidade-online/cnae/ - https://concla.ibge.gov.br/busca-online-cnae.html
--------------------------------------------------------------------------------------------------------------------------------------------------------------
	AND cpf_cnpj.cnpj = {cnpj}
	AND chave.chave NOT IN (
		SELECT
			DISTINCT chave.chave
		FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_NFE AS nfe -- Juntando com a tabela NFE para, a partir dela, fazer ligação com a tabela DIM_DESTINATARIO_EMITENTE

		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE AS chave -- Juntando com tabela DIM_CHAVE para pegar a chave da nota fiscal
			ON nfe.id_chave = chave.id_chave

		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS emitente_destinatario -- Relação das notas fiscais com os dados de seus emitentes
			ON nfe.id_emitente = emitente_destinatario.id_destinatario_emitente

		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CPF_CNPJ AS cpf_cnpj -- Relação com tabela DIM_CPF_CNPJ para pegar o cnpj
			ON emitente_destinatario.id_cpf_cnpj = cpf_cnpj.id_cpf_cnpj

		WHERE
			nfe.nota_cancelada = 1 AND
			cpf_cnpj.cnpj = {cnpj}
	)
ORDER BY chave.chave;
"""

### Resultados

In [132]:
notas_ativas_raw = busca_sql(query_notas_ativas_raw)
len(notas_ativas_raw["chave"].values)

48

In [133]:
notas_ativas_filtered = busca_sql(query_notas_ativas_filtered)
len(notas_ativas_filtered["chave"].values)

48

In [134]:
nao_passou_no_filtro = comparando_dfs(notas_ativas_raw["chave"].values, notas_ativas_filtered["chave"].values)
nao_passou_no_filtro, len(nao_passou_no_filtro)

([], 0)

## Notas Devolução

### Queries

In [135]:
query_notas_ativas_raw_devolucao = \
rf"""
SELECT
	DISTINCT chave.chave
FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_NFE AS nfe -- Juntando com a tabela NFE para, a partir dela, fazer ligação com a tabela DIM_DESTINATARIO_EMITENTE
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE AS chave -- Juntando com tabela DIM_CHAVE para pegar a chave da nota fiscal
	ON nfe.id_chave = chave.id_chave
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS emitente_destinatario -- Relação das notas fiscais com os dados de seus emitentes
	ON nfe.id_emitente = emitente_destinatario.id_destinatario_emitente
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CPF_CNPJ AS cpf_cnpj -- Relação com tabela DIM_CPF_CNPJ para pegar o cnpj
	ON emitente_destinatario.id_cpf_cnpj = cpf_cnpj.id_cpf_cnpj
WHERE
	nfe.nota_cancelada = 0 AND
	nfe.id_finalidade = 4 AND
	cpf_cnpj.cnpj = {cnpj} AND
	chave.chave NOT IN (
		SELECT
			DISTINCT chave.chave
		FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_NFE AS nfe -- Juntando com a tabela NFE para, a partir dela, fazer ligação com a tabela DIM_DESTINATARIO_EMITENTE
		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE AS chave -- Juntando com tabela DIM_CHAVE para pegar a chave da nota fiscal
			ON nfe.id_chave = chave.id_chave
		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS emitente_destinatario -- Relação das notas fiscais com os dados de seus emitentes
			ON nfe.id_emitente = emitente_destinatario.id_destinatario_emitente
		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CPF_CNPJ AS cpf_cnpj -- Relação com tabela DIM_CPF_CNPJ para pegar o cnpj
			ON emitente_destinatario.id_cpf_cnpj = cpf_cnpj.id_cpf_cnpj
		WHERE
			nfe.nota_cancelada = 1 AND
			cpf_cnpj.cnpj = {cnpj}
	)
ORDER BY chave.chave;
"""

In [136]:
query_notas_ativas_filtered_devolucao = \
rf"""
SELECT
	DISTINCT chave.chave
FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_PRODUTO AS produtos -- Carregando a tabela produtos que é o alvo da query
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.FATO_NFE AS nfe -- Juntando com a tabela NFE para, a partir dela, fazer ligação com a tabela DIM_DESTINATARIO_EMITENTE
	ON produtos.id_chave = nfe.id_chave
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE AS chave -- Juntando com tabela DIM_CHAVE para pegar a chave da nota fiscal
	ON produtos.id_chave = chave.id_chave
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS emitente -- Relação das notas fiscais com os dados de seus emitentes
	ON nfe.id_emitente = emitente.id_destinatario_emitente
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CPF_CNPJ AS cpf_cnpj -- Relação com tabela DIM_CPF_CNPJ para pegar o cnpj
	ON emitente.id_cpf_cnpj = cpf_cnpj.id_cpf_cnpj
INNER JOIN RF.dbo.t_pessoa_juridica AS pessoa_juridica -- Relação com banco de dados da Rceita Federal através do NPJ para filtras os códigos CNAE das empresas
	ON cpf_cnpj.cnpj = pessoa_juridica.cnpj
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_REFERENCIA AS referencia -- Relação para pegar referência
	ON nfe.id_chave = referencia.id_chave
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE_REFERENCIADA AS chave_referenciada -- Relação para pegar a chave da NF que está sendo referenciada
	ON referencia.id_chave_referenciada = chave_referenciada.id_chave_referenciada
--------------------------------------------------------------------------------------------------------------------------------------------------------------

WHERE
--FILTRO PRODUTOS
	(
		-- Código para bebidas, líquidos alcoólicos e vinagres (Capítulo 22)
		produtos.codigo_ncm LIKE '22071%' OR
		produtos.codigo_ncm LIKE '22072%' OR
		-- Códigos NCM para produtos farmacêuticos (Capítulo 30)
		produtos.codigo_ncm LIKE '30%' OR
		-- Código Extratos tanantes e tintoriais (Capítulo 32)
		produtos.codigo_ncm LIKE '320416%' OR -- corantes reagentes e preparações à base desses corantes
		-- Código para produtos para fotografia e cinematografia (Capítulo 37)
		produtos.codigo_ncm LIKE '370110%' OR -- filme para raio x
		-- Código produtos químicos diversos (Capítulo 38)
		produtos.codigo_ncm LIKE '3821%' OR -- Meios de cultura preparados para o desenvolvimento e a manutenção de microrganismos 
		produtos.codigo_ncm LIKE '3822%' OR -- reagentes de diagnóstico ou de laboratório (e.g. analisador de PH, glicose liquida, fósforo UV, hemoglobina glicada, acido urico)
		-- Código NCM para produtos de borracha (Capítulo 40)
		produtos.codigo_ncm LIKE '40151%' OR
		-- Código para tecidos especiais, tecidos tufados, rendas, tapeçarias, passamanarias, bordados (Capítulo 58)
		produtos.codigo_ncm LIKE '5803%' OR -- Tecidos em ponto de gaze
		-- Código para Outros artefatos têxteis confeccionados (Capítulo 60)
		produtos.codigo_ncm LIKE '6307901%' OR -- Outros artefatos confeccionados de falso tecido (e.g. máscara)
		-- Código Reatores nucleares, caldeiras, máquinas, aparelhos e instrumentos mecânicos, e suas partes (Capítulo 84)
		produtos.codigo_ncm LIKE '842119%' OR -- Centrifugadores para laboratórios de análises, ensaios ou pesquisas científicas
		produtos.codigo_ncm LIKE '842191%' OR -- Partes de centrifugadores
		produtos.codigo_ncm LIKE '84798912' OR -- Distribuidores e doseadores de sólidos ou de líquidos  (e.g. micropipeta)
		-- Códigos NCM para instrumentos e aparelhos de óptica, de fotografia, de cinematografia, de medida, de controle ou de precisão.
		-- instrumentos e aparelhos médico-cirúrgicos suas partes e acessórios (Capítulo 90)
		produtos.codigo_ncm LIKE '9011%' OR --  Microscópios ópticos, incluindo os microscópios para fotomicrografia, cinefotomicrografia ou microprojeção.
		produtos.codigo_ncm LIKE '9012%' OR --  Microscópios, exceto ópticos; difratógrafos.
		produtos.codigo_ncm LIKE '9018%' OR -- instumentos e aparelhos para medicina, cirurgia, odontologia e veterinária
		produtos.codigo_ncm LIKE '9019%' OR -- aparelhos de mecanoterapia, aparelhos de massagem, de psicotécnica, de ozonoterapia...
		produtos.codigo_ncm LIKE '9020%' OR -- outros aparelhos respiratórios e máscaras contra gases
		produtos.codigo_ncm LIKE '9021%' OR -- Artigos e aparelhos ortopédicos
		produtos.codigo_ncm LIKE '9022%' OR -- Aparelhos de raios X e aparelhos que utilizem radiações alfa
		produtos.codigo_ncm LIKE '9027%' OR -- Instrumentos e aparelhos para análises físicas ou químicas
		-- Códigos para Obras diversas (Capítulo 96)
		produtos.codigo_ncm LIKE '9619%' OR --  Absorventes e tampões higiênicos, cueiros e fraldas para bebês e artigos higiênicos semelhantes, de qualquer matéria.
			-- FONTE: http://www5.sefaz.mt.gov.br/documents/6071037/6401784/Tabela+NCM+-+MDIC+atualizada.pdf/bc780e4b-fd2f-4312-879c-65d5fd1ff49d / Pág. 129
		produtos.id_medicamento != -1 -- possui id de medicamento válido
	) AND
--------------------------------------------------------------------------------------------------------------------------------------------------------------
--FILTRO NOTA FISCAL
	nfe.nota_cancelada = 0 AND -- Notas que não foram canceladas
	nfe.id_modelo = 2 AND -- Notas 55
	nfe.id_finalidade = 4 AND -- Notas que possuem valor 4 (DEVOLUCAO DE MERCADORIA)
	chave_referenciada.chave_nf IS NOT NULL AND
--------------------------------------------------------------------------------------------------------------------------------------------------------------
--FILTRO PESSOA JURIDICA
	pessoa_juridica.cnae_fiscal IN (2110600, 2121101, 2121102, 2121103, 2123800, 2312500, 3299099, 4618401,
									4644301, 4645101, 4664800, 4684299, 4771701, 4771702, 4771703, 4773300) -- Empresas farmoquimicas e farmaceuticas
-- Fonte: https://www.contabilizei.com.br/contabilidade-online/cnae/ - https://concla.ibge.gov.br/busca-online-cnae.html
--------------------------------------------------------------------------------------------------------------------------------------------------------------
	AND cpf_cnpj.cnpj = {cnpj}
	AND chave.chave NOT IN (
		SELECT
			DISTINCT chave.chave
		FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_NFE AS nfe -- Juntando com a tabela NFE para, a partir dela, fazer ligação com a tabela DIM_DESTINATARIO_EMITENTE

		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE AS chave -- Juntando com tabela DIM_CHAVE para pegar a chave da nota fiscal
			ON nfe.id_chave = chave.id_chave

		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS emitente_destinatario -- Relação das notas fiscais com os dados de seus emitentes
			ON nfe.id_emitente = emitente_destinatario.id_destinatario_emitente

		INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CPF_CNPJ AS cpf_cnpj -- Relação com tabela DIM_CPF_CNPJ para pegar o cnpj
			ON emitente_destinatario.id_cpf_cnpj = cpf_cnpj.id_cpf_cnpj

		WHERE
			nfe.nota_cancelada = 1 AND
			cpf_cnpj.cnpj = {cnpj}
	)
"""

### Resultados

In [137]:
notas_ativas_raw_devolucao = busca_sql(query_notas_ativas_raw_devolucao)
len(notas_ativas_raw_devolucao["chave"].values)

3

In [138]:
notas_ativas_filtered_devolucao = busca_sql(query_notas_ativas_filtered_devolucao)
len(notas_ativas_filtered_devolucao["chave"].values)

3

In [139]:
nao_passou_no_filtro_devolucao = comparando_dfs(notas_ativas_raw_devolucao["chave"].values, notas_ativas_filtered_devolucao["chave"].values)
nao_passou_no_filtro_devolucao, len(nao_passou_no_filtro_devolucao)

([], 0)

## Notas Canceladas

### Queries

In [140]:
query_notas_raw_cancelada = \
rf"""
SELECT
	DISTINCT chave.chave
FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_NFE AS nfe -- Juntando com a tabela NFE para, a partir dela, fazer ligação com a tabela DIM_DESTINATARIO_EMITENTE
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE AS chave -- Juntando com tabela DIM_CHAVE para pegar a chave da nota fiscal
	ON nfe.id_chave = chave.id_chave
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS emitente_destinatario -- Relação das notas fiscais com os dados de seus emitentes
	ON nfe.id_emitente = emitente_destinatario.id_destinatario_emitente
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CPF_CNPJ AS cpf_cnpj -- Relação com tabela DIM_CPF_CNPJ para pegar o cnpj
	ON emitente_destinatario.id_cpf_cnpj = cpf_cnpj.id_cpf_cnpj
WHERE
	nfe.nota_cancelada = 1 AND
	cpf_cnpj.cnpj = {cnpj}
"""

In [141]:
query_notas_filtered_cancelada = \
rf"""
SELECT
	DISTINCT chave.chave
FROM NFE_PUBLICA_PRODUCAO.dbo.FATO_PRODUTO AS produtos -- Carregando a tabela produtos que é o alvo da query
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.FATO_NFE AS nfe -- Juntando com a tabela NFE para, a partir dela, fazer ligação com a tabela DIM_DESTINATARIO_EMITENTE
	ON produtos.id_chave = nfe.id_chave
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CHAVE AS chave -- Juntando com tabela DIM_CHAVE para pegar a chave da nota fiscal
	ON produtos.id_chave = chave.id_chave
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_DESTINATARIO_EMITENTE AS emitente -- Relação das notas fiscais com os dados de seus emitentes
	ON nfe.id_emitente = emitente.id_destinatario_emitente
INNER JOIN NFE_PUBLICA_PRODUCAO.dbo.DIM_CPF_CNPJ AS cpf_cnpj -- Relação com tabela DIM_CPF_CNPJ para pegar o cnpj
	ON emitente.id_cpf_cnpj = cpf_cnpj.id_cpf_cnpj
INNER JOIN RF.dbo.t_pessoa_juridica AS pessoa_juridica -- Relação com banco de dados da Rceita Federal através do NPJ para filtras os códigos CNAE das empresas
	ON cpf_cnpj.cnpj = pessoa_juridica.cnpj
--------------------------------------------------------------------------------------------------------------------------------------------------------------
WHERE
--FILTRO PRODUTOS
	(
		-- Código para bebidas, líquidos alcoólicos e vinagres (Capítulo 22)
		produtos.codigo_ncm LIKE '22071%' OR
		produtos.codigo_ncm LIKE '22072%' OR
		-- Códigos NCM para produtos farmacêuticos (Capítulo 30)
		produtos.codigo_ncm LIKE '30%' OR
		-- Código Extratos tanantes e tintoriais (Capítulo 32)
		produtos.codigo_ncm LIKE '320416%' OR -- corantes reagentes e preparações à base desses corantes
		-- Código para produtos para fotografia e cinematografia (Capítulo 37)
		produtos.codigo_ncm LIKE '370110%' OR -- filme para raio x
		-- Código produtos químicos diversos (Capítulo 38)
		produtos.codigo_ncm LIKE '3821%' OR -- Meios de cultura preparados para o desenvolvimento e a manutenção de microrganismos 
		produtos.codigo_ncm LIKE '3822%' OR -- reagentes de diagnóstico ou de laboratório (e.g. analisador de PH, glicose liquida, fósforo UV, hemoglobina glicada, acido urico)
		-- Código NCM para produtos de borracha (Capítulo 40)
		produtos.codigo_ncm LIKE '40151%' OR
		-- Código para tecidos especiais, tecidos tufados, rendas, tapeçarias, passamanarias, bordados (Capítulo 58)
		produtos.codigo_ncm LIKE '5803%' OR -- Tecidos em ponto de gaze
		-- Código para Outros artefatos têxteis confeccionados (Capítulo 60)
		produtos.codigo_ncm LIKE '6307901%' OR -- Outros artefatos confeccionados de falso tecido (e.g. máscara)
		-- Código Reatores nucleares, caldeiras, máquinas, aparelhos e instrumentos mecânicos, e suas partes (Capítulo 84)
		produtos.codigo_ncm LIKE '842119%' OR -- Centrifugadores para laboratórios de análises, ensaios ou pesquisas científicas
		produtos.codigo_ncm LIKE '842191%' OR -- Partes de centrifugadores
		produtos.codigo_ncm LIKE '84798912' OR -- Distribuidores e doseadores de sólidos ou de líquidos  (e.g. micropipeta)
		-- Códigos NCM para instrumentos e aparelhos de óptica, de fotografia, de cinematografia, de medida, de controle ou de precisão.
		-- instrumentos e aparelhos médico-cirúrgicos suas partes e acessórios (Capítulo 90)
		produtos.codigo_ncm LIKE '9011%' OR --  Microscópios ópticos, incluindo os microscópios para fotomicrografia, cinefotomicrografia ou microprojeção.
		produtos.codigo_ncm LIKE '9012%' OR --  Microscópios, exceto ópticos; difratógrafos.
		produtos.codigo_ncm LIKE '9018%' OR -- instumentos e aparelhos para medicina, cirurgia, odontologia e veterinária
		produtos.codigo_ncm LIKE '9019%' OR -- aparelhos de mecanoterapia, aparelhos de massagem, de psicotécnica, de ozonoterapia...
		produtos.codigo_ncm LIKE '9020%' OR -- outros aparelhos respiratórios e máscaras contra gases
		produtos.codigo_ncm LIKE '9021%' OR -- Artigos e aparelhos ortopédicos
		produtos.codigo_ncm LIKE '9022%' OR -- Aparelhos de raios X e aparelhos que utilizem radiações alfa
		produtos.codigo_ncm LIKE '9027%' OR -- Instrumentos e aparelhos para análises físicas ou químicas
		-- Códigos para Obras diversas (Capítulo 96)
		produtos.codigo_ncm LIKE '9619%' OR --  Absorventes e tampões higiênicos, cueiros e fraldas para bebês e artigos higiênicos semelhantes, de qualquer matéria.
			-- FONTE: http://www5.sefaz.mt.gov.br/documents/6071037/6401784/Tabela+NCM+-+MDIC+atualizada.pdf/bc780e4b-fd2f-4312-879c-65d5fd1ff49d / Pág. 129
		produtos.id_medicamento != -1 -- possui id de medicamento válido
	) AND
--------------------------------------------------------------------------------------------------------------------------------------------------------------
--FILTRO NOTA FISCAL
	nfe.nota_cancelada = 1 AND -- Notas que foram canceladas
--------------------------------------------------------------------------------------------------------------------------------------------------------------
--FILTRO PESSOA JURIDICA 4645101
	pessoa_juridica.cnae_fiscal IN (2110600, 2121101, 2121102, 2121103, 2123800, 2312500, 3299099, 4618401,
									4644301, 4645101, 4664800, 4684299, 4771701, 4771702, 4771703, 4773300) -- Empresas farmoquimicas e farmaceuticas
-- Fonte: https://www.contabilizei.com.br/contabilidade-online/cnae/ - https://concla.ibge.gov.br/busca-online-cnae.html
--------------------------------------------------------------------------------------------------------------------------------------------------------------
	AND cpf_cnpj.cnpj = {cnpj}
"""

### Resultados

In [142]:
notas_raw_cancelada = busca_sql(query_notas_raw_cancelada)
len(notas_raw_cancelada["chave"].values)

5

In [143]:
notas_filtered_cancelada = busca_sql(query_notas_filtered_cancelada)
len(notas_filtered_cancelada["chave"].values)

5

In [144]:
nao_passou_no_filtro_cancelada = comparando_dfs(notas_raw_cancelada["chave"].values, notas_filtered_cancelada["chave"].values)
nao_passou_no_filtro_cancelada, len(nao_passou_no_filtro_cancelada)

([], 0)